In [1]:
import numpy as np 
from scipy.integrate import odeint
import os, sys, warnings
from pathlib import Path
from os.path import dirname, realpath
script_dir = Path(dirname(realpath('.')))
module_dir = str(script_dir)
sys.path.insert(0, module_dir + '/modules')
import utility as ut
import surrogate_nn as srnn
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch import nn
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, TensorDataset
warnings.filterwarnings('ignore')

In [2]:
DTYPE = 'float64'
L63_data_path = '../data/L63-trajectories'
save_folder='../data/adaptive-rate'
log_interval = 100
milestones = [10*2**n for n in range(15)]
learning_rate = 1e-3
drop = 0.7
steps = int(1e5)
save_interval = 100
N = 2000
L0 = 0.4
L1 = 3.5
beta = 4e-5
min_change = -(1.+0.1)**(1./log_interval) + 1.
train = np.load(f'{L63_data_path}/train.npy').astype(DTYPE)
test = np.load(f'{L63_data_path}/test.npy')[:, :, :1000].astype(DTYPE)

model = srnn.SurrogateModel_NN(3, 100, name='nn', save_folder=save_folder)
model.learn(train[:, :N], steps, 1e-3, batch_size='GD', log_interval=100, save_interval=100,\
           max_delta=0.5, min_change=min_change)
tau_f_rmse, tau_f_se, rmse, se = model.compute_tau_f(test[:100], error_threshold=0.05)
tau_f_1 = tau_f_se.mean()
tau_f_1

step: 0 loss: 1559983.287651 time: 0.0024 lr: 0.001000,  change: nan%
step: 100 loss: 743180.360166 time: 0.1112 lr: 0.001000,  change: -52.36%
step: 200 loss: 369648.112960 time: 0.2172 lr: 0.001000,  change: -50.26%
step: 300 loss: 232227.256843 time: 0.3259 lr: 0.001000,  change: -37.18%
step: 400 loss: 173989.701183 time: 0.4325 lr: 0.001000,  change: -25.08%
step: 500 loss: 135104.053664 time: 0.5394 lr: 0.001000,  change: -22.35%
step: 600 loss: 105050.544273 time: 0.6459 lr: 0.001000,  change: -22.24%
step: 700 loss: 79363.044066 time: 0.7533 lr: 0.001000,  change: -24.45%
step: 800 loss: 57705.904160 time: 0.8601 lr: 0.001000,  change: -27.29%
step: 900 loss: 41335.835422 time: 0.9676 lr: 0.001000,  change: -28.37%
step: 1000 loss: 29331.568894 time: 1.0734 lr: 0.001000,  change: -29.04%
step: 1100 loss: 21275.505888 time: 1.1793 lr: 0.001000,  change: -27.47%
step: 1200 loss: 15814.806600 time: 1.2850 lr: 0.001000,  change: -25.67%
step: 1300 loss: 12144.278121 time: 1.3910 lr

KeyboardInterrupt: 

In [ ]:
(1+0.01)**(0.01)-1

In [ ]:
min_change

In [ ]:
a = torch.rand(2, 2)
a

In [ ]:
torch.sum(a**2)

In [ ]:
torch.norm(a)**2